# Trader

## Real

1 - Pergunta à estratégia quantas amostras, de qual 'candle_interval', no mínimo, são necessárias para o cálculo da posição

2 - Solicita as klines correspondentes

3 - Envia as klines e o 'Asset.info()' ao módulo de estratégia para análise, recolhendo o sinal correspondente

4 - Para sinal diferente de 'hold', aciona o 'order_handler'

5 - Dorme

6 - 'Refresca' as klines

7 - Repete a partir do passo 3

## Backing Test

1 - Pede todas as klines

2 - Pergunta à estratégia quantas amostras, de qual 'candle_interval', no mínimo, são necessárias para o cálculo da posição

3 - Envia a fatia (suficientemente longa) de klines mais antiga e o 'Asset.info()' ao módulo de estratégia para análise, recolhendo o sinal correspondente

4 - Para sinal diferente de 'hold' procede a 'ordem fictícia'

5 - 'Avança uma' kline, 'refrescando' a fatia

6 - Repete a partir do passo 3

## Observações

1 - É a estratégia que instancia o stop  
2 - Existe um problema, ainda sem causa descoberta: o módulo 'klines_handler' está retornando uma kline a menos  
    
    2.1 - Solução sugerida:  
    
    Em OneMinuteCandlesAmount.total(self, out_candle_number), 'pedir um out_candle_number a mais, truncando em 
    'get_latest' para o número de candles desejado

In [1]:
from modules.klines_handler import *
from modules.assets_handler import AssetsHandler
from pydoc import locate

##Database credentials
DB_HOST = 'localhost'
DB_PORT = 5432
DB_NAME = 'pandazord_database'                      
DB_USER = 'pandazord'
DB_PASSWORD = 'QBBV9E%pcYKHUcjj'

## Market and Asset
exchange_name = 'binance'
asset_symbol = 'BTCUSDT'

In [2]:
Asset = AssetsHandler(DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME, exchange_name, asset_symbol)
op_param = Asset.info().iloc[0].operational_parameters

In [20]:
sleep_time_seconds = (OneMinuteCandlesAmount(op_param['sleep_duration']).per_out_candle)*60
strategy_name = op_param['strategy']['name']
stop_name = op_param['stop_loss']['name']

Strategy = locate('modules.strategies.' + strategy_name)(op_param)
#StopLoss = locate('modules.stop_loss.' + stop_name)(op_param)

In [21]:
print (sleep_time_seconds)

60


In [24]:
OneMinuteCandlesAmount(op_param['candle_interval']).per_out_candle

720

In [ ]:
op_param[]